In [7]:
from utils import ModelImporter, Preprocessing
import torch
import pandas as pd
import numpy as np
from sklearn import metrics

# Analize Test Data Results with model train with entire data and tuned parameters

In [2]:
pre = Preprocessing('fer2013')
pre.load_data(filename='test_public_norm.csv', name='test')

X_df = pre.get(name='test').drop(columns=['emotion'])
y_df = pre.get(name='test')['emotion']

In [3]:
dtype = torch.float
device = torch.device("cpu")

n_classes = 7
n_epochs = 100
learning_rate = 0.0001
batch_size = 32
epoch_n=80

model_name = f'cnn_simple_{learning_rate}_{batch_size}_{n_epochs}_{n_classes}_epoch{epoch_n}'

m_importer = ModelImporter('fer2013')

model = m_importer.load_nn_model(model_name)

['cnn_simple_0.0001_32_100_7_epoch80', 1, 7, [48, 48], 15, 2]
load model CnnSimple(
  (drop_hidden): Dropout(p=0.5)
  (drop_visible): Dropout(p=0.2)
  (conv1): Conv2d(1, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm1): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8640, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=7, bias=True)
)


In [8]:
X_test = model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_test = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [9]:
y_pred_prob = model(X_test)

In [10]:
y_pred = y_pred_prob.argmax(1)

In [11]:
accuracy_soft = (y_pred == y_test).float().mean()
accuracy_soft

tensor(0.4806)

In [17]:
y_pred

tensor([2, 3, 2,  ..., 4, 0, 6])

In [18]:
results = pd.DataFrame()

In [19]:
results['y_test'] = y_test
results['y_pred'] = y_pred
results['right'] = (y_pred == y_test).float()
results['ones'] = 1

In [20]:
results.tail()

,y_test,y_pred,right,ones
3584,4,0,0.0,1
3585,3,3,1.0,1
3586,4,4,1.0,1
3587,4,0,0.0,1
3588,4,6,0.0,1


In [21]:
report = results.groupby('y_test').sum()
report

,y_pred,right,ones
y_test,,,
0,1314,126.0,467
1,118,11.0,56
2,1620,164.0,496
3,2941,617.0,895
4,2307,243.0,653
5,1785,272.0,415
6,2672,292.0,607


In [22]:
report['%'] = report['right']/report['ones']
report

,y_pred,right,ones,%
y_test,,,,
0,1314,126.0,467,0.269807
1,118,11.0,56,0.196429
2,1620,164.0,496,0.330645
3,2941,617.0,895,0.689385
4,2307,243.0,653,0.372129
5,1785,272.0,415,0.655422
6,2672,292.0,607,0.481054


In [23]:
report['ones'].sum()

3589

In [24]:
#class 3 is somhow well recognized (also is the class with more samples) 
#whereas not a single sample of class 2 was properly detected

# Analize Test Data Results with model train with disgust only expanded data 

In [26]:
epoch_n=80
model_name = f'cnn_simple_{learning_rate}_{batch_size}_{n_epochs}_{n_classes}_epoch{epoch_n}'
m_importer = ModelImporter('fer2013_expanded_disgust')
model = m_importer.load_nn_model(model_name)

['cnn_simple_0.0001_32_100_7_epoch80', 1, 7, [48, 48], 15, 2]
load model CnnSimple(
  (drop_hidden): Dropout(p=0.5)
  (drop_visible): Dropout(p=0.2)
  (conv1): Conv2d(1, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm1): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8640, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=7, bias=True)
)


In [27]:
X_test = model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_test = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [28]:
y_pred_prob = model(X_test)

In [29]:
y_pred = y_pred_prob.argmax(1)

In [30]:
accuracy_soft = (y_pred == y_test).float().mean()
accuracy_soft

tensor(0.4987)

In [31]:
results_expanded_disgust = pd.DataFrame()

In [32]:
results_expanded_disgust['y_test'] = y_test
results_expanded_disgust['y_pred'] = y_pred
results_expanded_disgust['right'] = (y_pred == y_test).float()
results_expanded_disgust['ones'] = 1

In [33]:
results_expanded_disgust.tail()

,y_test,y_pred,right,ones
3584,4,0,0.0,1
3585,3,3,1.0,1
3586,4,4,1.0,1
3587,4,0,0.0,1
3588,4,6,0.0,1


In [34]:
report_exp_disgust = results_expanded_disgust.groupby('y_test').sum()
report_exp_disgust

,y_pred,right,ones
y_test,,,
0,1211,146.0,467
1,101,19.0,56
2,1649,135.0,496
3,2874,680.0,895
4,2312,247.0,653
5,1818,275.0,415
6,2657,288.0,607


In [35]:
report_exp_disgust['%'] = report_exp_disgust['right']/report_exp_disgust['ones']
report_exp_disgust

,y_pred,right,ones,%
y_test,,,,
0,1211,146.0,467,0.312634
1,101,19.0,56,0.339286
2,1649,135.0,496,0.272177
3,2874,680.0,895,0.759777
4,2312,247.0,653,0.378254
5,1818,275.0,415,0.662651
6,2657,288.0,607,0.474465


In [ ]:
# the general accuracy has improved from 46.81% to 49.23% 
# the main changed has happened in the prediction of the disgust class, going from 14.28% to 23.21% 
# this considerable improvement has happened only by adding the flippedimages 

## Analize Test Data Results with model train with expanded data 

In [52]:
dtype = torch.float
device = torch.device("cpu")

n_classes = 7
n_epochs = 100
learning_rate = 0.0001
batch_size = 32
epoch_n=60

In [53]:
model_name = f'cnn_simple_{learning_rate}_{batch_size}_{n_epochs}_{n_classes}_epoch{epoch_n}'
m_importer = ModelImporter('fer2013_expanded')
model = m_importer.load_nn_model(model_name)
print(model.state_dict())

['cnn_simple_expanded_0.0001_32_100_7_epoch60', 1, 7, [48, 48], 15, 2]
load model CnnSimple(
  (drop_hidden): Dropout(p=0.5)
  (drop_visible): Dropout(p=0.2)
  (conv1): Conv2d(1, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm1): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8640, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=7, bias=True)
)
OrderedDict([('conv1.weight', tensor([[[[ 0.0042, -0.1761,  0.1045],
          [-0.2366,  0.2980,  0.0053],
          [ 0.2086, -0.0152, -0.1142]]],


        [[[ 0.0581,  0.1283,  0.2127],
          [-0.0529, -0.3404, -0.3195],
          [ 0.0550,  0.2163,  0.1197]]],


        [[[ 0.2406, -0.0417, -0.0047],
          [ 0.3672,  0.1402,  0.2999],
          [ 0.1990,  0.0204,  0.2339]]],


        [[[-0.0999,  0.3016, -0.0296],
          [ 0.2190,  0.2398, -

In [54]:
X_test = model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_test = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [55]:
y_pred_prob = model(X_test)

In [56]:
y_pred = y_pred_prob.argmax(1)

In [57]:
accuracy_soft = (y_pred == y_test).float().mean()
accuracy_soft

tensor(0.5043)

In [58]:
results_expanded = pd.DataFrame()

In [59]:
results_expanded['y_test'] = y_test
results_expanded['y_pred'] = y_pred
results_expanded['right'] = (y_pred == y_test).float()
results_expanded['ones'] = 1

In [60]:
results_expanded.tail()

,y_test,y_pred,right,ones
3584,4,3,0.0,1
3585,3,3,1.0,1
3586,4,6,0.0,1
3587,4,0,0.0,1
3588,4,6,0.0,1


In [61]:
report_exp = results_expanded.groupby('y_test').sum()
report_exp

,y_pred,right,ones
y_test,,,
0,1346,132.0,467
1,130,9.0,56
2,1766,122.0,496
3,2971,639.0,895
4,2537,312.0,653
5,1852,290.0,415
6,2770,306.0,607


In [62]:
report_exp['%'] = report_exp['right']/report_exp['ones']
report_exp

,y_pred,right,ones,%
y_test,,,,
0,1346,132.0,467,0.282655
1,130,9.0,56,0.160714
2,1766,122.0,496,0.245968
3,2971,639.0,895,0.713966
4,2537,312.0,653,0.477795
5,1852,290.0,415,0.698795
6,2770,306.0,607,0.504119
